In [2]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [3]:
# 한국어 감성 분석을 위한 KFP-BERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
model = BertForSequenceClassification.from_pretrained("beomi/kcbert-base")

# 모델을 평가 모드로 설정
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
data=pd.read_csv('20230101_to_20230331_news_data.csv') #각자 자기가 맡은 파일이름 쓰면 됨

In [12]:
# 데이터프레임을 10개의 부분으로 나누기 (각자 원하는 만큼 데이터 쪼개기
splits = np.array_split(data, 10)

# 각 부분을 다른 변수에 할당
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10= splits
#나는 10개로 했으니까 10개의 데이터프레임에 할당


C:\Users\USER\Anaconda_\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [13]:

# 한국어 감성 분석을 위한 KFP-BERT 모델 및 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
model = BertForSequenceClassification.from_pretrained("beomi/kcbert-base")

# 모델을 평가 모드로 설정
model.eval()

# 데이터프레임 리스트를 생성 (예제로 df1, df2, ..., df50)
dataframes = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]

# 모든 데이터프레임에 대한 감성 분석 수행
for index, df in enumerate(dataframes, 1):
    sentiment_scores = []
    for content in df['title']:
        inputs = tokenizer(content, return_tensors="pt", truncation=True, padding=True, max_length=300)  # max_length를 300으로 변경
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        positive_prob = probs[:, 1]
        negative_prob = probs[:, 0]

        # 문서별 감성 점수 계산 및 저장
        for i in range(positive_prob.size(0)):
            sentiment_score = (positive_prob[i] - negative_prob[i]).item()
            sentiment_scores.append(sentiment_score)

            if sentiment_score > 0:
                print(f"긍정적인 내용입니다. 감성 점수: {sentiment_score}")
            else:
                print(f"부정적인 내용입니다. 감성 점수: {sentiment_score}")

    # 감성 점수 리스트를 현재 데이터프레임에 추가
    df['sentiment_score'] = sentiment_scores
    print(f"DataFrame {index}의 감성 분석이 완료되었습니다.")

    # 데이터프레임을 CSV 파일로 저장
    df.to_csv(f'DataFrame{index}_sentiment_analysis.csv', index=False)
    print(f"DataFrame {index} 저장 완료: DataFrame{index}_sentiment_analysis.csv")

    # 이미 생성된 데이터프레임들을 리스트에 저장
dataframes = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]

# 모든 데이터프레임을 행 기준으로 결합
combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv('df_한종민.csv', index=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


부정적인 내용입니다. 감성 점수: -0.02982485294342041
부정적인 내용입니다. 감성 점수: -0.06905964016914368
부정적인 내용입니다. 감성 점수: -0.031663715839385986
부정적인 내용입니다. 감성 점수: -0.007264494895935059
부정적인 내용입니다. 감성 점수: -0.046455204486846924
긍정적인 내용입니다. 감성 점수: 0.03475669026374817
긍정적인 내용입니다. 감성 점수: 0.019938141107559204
부정적인 내용입니다. 감성 점수: -0.01933002471923828
긍정적인 내용입니다. 감성 점수: 0.021558433771133423
부정적인 내용입니다. 감성 점수: -0.08396327495574951
부정적인 내용입니다. 감성 점수: -0.2708803117275238
부정적인 내용입니다. 감성 점수: -0.10161316394805908
부정적인 내용입니다. 감성 점수: -0.3017483353614807
부정적인 내용입니다. 감성 점수: -0.09613797068595886
부정적인 내용입니다. 감성 점수: -0.10811972618103027
부정적인 내용입니다. 감성 점수: -0.05420076847076416
긍정적인 내용입니다. 감성 점수: 0.013888746500015259
긍정적인 내용입니다. 감성 점수: 0.08484375476837158
부정적인 내용입니다. 감성 점수: -0.06580784916877747
부정적인 내용입니다. 감성 점수: -0.004685759544372559
부정적인 내용입니다. 감성 점수: -0.053355127573013306
부정적인 내용입니다. 감성 점수: -0.05318409204483032
부정적인 내용입니다. 감성 점수: -0.05570301413536072
부정적인 내용입니다. 감성 점수: -0.042291998863220215
부정적인 내용입니다. 감성 점수: -0.1291037201881408

In [14]:

# 이미 생성된 데이터프레임들을 리스트에 저장
dataframes = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]

# 모든 데이터프레임을 행 기준으로 결합
combined_df = pd.concat(dataframes, ignore_index=True)

# 결과 데이터프레임 확인
print(combined_df)

# 결과 데이터


           date                                           title  \
0      20230101          “지금 내놔봐야 미분양 굴욕”...1월 분양 작년 3분의 1로 ‘뚝’   
1      20230101                원희룡 “미분양 대책 준비 다 됐다…규제지역 해제 고민중”   
2      20230101            [일요와이드] 집값 내리막·깡통전세 우려 확산…2023년 전망은?   
3      20230101                 윤세영 태영그룹 창립회장 "어려울수록 원칙·정직 지켜야"   
4      20230101                   평소 찜해둔 아파트 급매로 떴는데…“아직 때 아니다”   
...         ...                                             ...   
26597  20230331                    집 짓기 꺼리는 건설사들… 3년 뒤 집값이 위험하다   
26598  20230331  [이미연의 발로 뛰는 부동산] "아직 괜찮지만…" 부산에 `미분양 경고등` 켜질까요   
26599  20230331       네옴시티 호재에도 해외건설 수주 주춤...중동서 늘은 물량 동남아서 빠졌다   
26600  20230331           삼성전자가 300조 투자하는 용인…처인구 아파트값 보름새 1억 상승   
26601  20230331              은마아파트 3.3㎡당 7100만원…분양가, 주변시세 추월 속출   

                                                 content   press  \
0      금리인상·청약부진 이어지자건설사들 “일단 미루고 보자”새해 첫달 분양 7000가구 ...    매일경제   
1      부동산 정책 조만간 발표할 듯건설노조 불법행위 근절 강조“상반기 노조횡포 바로잡을 ...   

In [15]:

# 데이터프레임을 CSV 파일로 저장
combined_df.to_csv('df_한종민완료.csv', index=False)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26607 entries, 0 to 26606
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     26607 non-null  object
 1   title    26602 non-null  object
 2   content  26603 non-null  object
 3   press    26557 non-null  object
dtypes: object(4)
memory usage: 831.6+ KB


In [10]:
data.isnull().sum()

date        0
title       5
content     4
press      50
dtype: int64

In [11]:
data=data.dropna(subset=['title'])
data.isnull().sum()

date        0
title       0
content     4
press      50
dtype: int64

In [17]:
data2=pd.read_csv('20230401_to_20230630_news.csv_data.csv')
data2

,date,title,content,press
0,20230401,느긋한 강남 아파트마저 웃었다…서울은 재건축의 천국? [부동산360],"도곡한신재건축추진위, 안전진단 신청 접수서울 아파트, 7월부터 구청이 안전진단비 융...",헤럴드경제
1,20230401,"1·3 대책 후 서울 첫 분양 영등포자이, 예비당첨자 계약 ‘완판’","GS건설의 영등포자이 디그니티 모델하우스를 살펴보는 방문객들. 연합뉴스 ""입지 여건...",문화일보
2,20230401,"‘규제완화 후 서울 첫 분양’ 영등포자이, 예비당첨자 계약에서 ‘완판’",영등포자이 디그니티 견본주택. GS건설 제공헤럴드경제=안대용 기자 정부의 1·3 부...,헤럴드경제
3,20230401,서울 신축 아파트가 8억대?…아껴둔 통장 지를까요?[부동산360],‘휘경자이 디센시아’ 견본주택 가보니31일 강남구 대치동에 위치한 ‘휘경자이 디센시...,헤럴드경제
4,20230401,"규제완화 후 서울 첫 분양…영등포자이, 예비당첨자 계약서 '완판'",1·3 부동산 대책 발표 이후 서울에서 처음으로 입주자 모집공고를 내 눈길을 끌었던...,한국경제TV
...,...,...,...,...
26859,20230630,"전국 아파트 가격, 14개월 만에 하락 멈춰","서울 아파트값은 6주 연속 상승세수도권 전셋값, 17개월 만에 반등지난해 5월부터 ...",동아일보
26860,20230630,"압구정 현대는 10억까지? 고가 단지 재건축, 부담금 더 는다","정부, 野 반대에 재건축 초과이익환수법 수정안 국회 제출서울 강남구 압구정동 현대아...",조선일보
26861,20230630,"[기업 브리핑] 태영건설, ‘대한민국 조경대상’ 국무총리상 외","태영건설, ‘대한민국 조경대상’ 국무총리상태영건설이 ‘대한민국 조경대상’에서 올해 ...",조선일보
26862,20230630,"용산 집값, 송파 제치고 3위…흔들리는 ‘강남 3구 아성’",서울 용산구 서빙고동 신동아 아파트 전용면적 166㎡(55평)가 지난달 36억700...,중앙일보


In [18]:
data2.isnull().sum()

date        0
title      11
content     3
press      42
dtype: int64

In [19]:
data2=data2.dropna(subset='title')
data2

,date,title,content,press
0,20230401,느긋한 강남 아파트마저 웃었다…서울은 재건축의 천국? [부동산360],"도곡한신재건축추진위, 안전진단 신청 접수서울 아파트, 7월부터 구청이 안전진단비 융...",헤럴드경제
1,20230401,"1·3 대책 후 서울 첫 분양 영등포자이, 예비당첨자 계약 ‘완판’","GS건설의 영등포자이 디그니티 모델하우스를 살펴보는 방문객들. 연합뉴스 ""입지 여건...",문화일보
2,20230401,"‘규제완화 후 서울 첫 분양’ 영등포자이, 예비당첨자 계약에서 ‘완판’",영등포자이 디그니티 견본주택. GS건설 제공헤럴드경제=안대용 기자 정부의 1·3 부...,헤럴드경제
3,20230401,서울 신축 아파트가 8억대?…아껴둔 통장 지를까요?[부동산360],‘휘경자이 디센시아’ 견본주택 가보니31일 강남구 대치동에 위치한 ‘휘경자이 디센시...,헤럴드경제
4,20230401,"규제완화 후 서울 첫 분양…영등포자이, 예비당첨자 계약서 '완판'",1·3 부동산 대책 발표 이후 서울에서 처음으로 입주자 모집공고를 내 눈길을 끌었던...,한국경제TV
...,...,...,...,...
26859,20230630,"전국 아파트 가격, 14개월 만에 하락 멈춰","서울 아파트값은 6주 연속 상승세수도권 전셋값, 17개월 만에 반등지난해 5월부터 ...",동아일보
26860,20230630,"압구정 현대는 10억까지? 고가 단지 재건축, 부담금 더 는다","정부, 野 반대에 재건축 초과이익환수법 수정안 국회 제출서울 강남구 압구정동 현대아...",조선일보
26861,20230630,"[기업 브리핑] 태영건설, ‘대한민국 조경대상’ 국무총리상 외","태영건설, ‘대한민국 조경대상’ 국무총리상태영건설이 ‘대한민국 조경대상’에서 올해 ...",조선일보
26862,20230630,"용산 집값, 송파 제치고 3위…흔들리는 ‘강남 3구 아성’",서울 용산구 서빙고동 신동아 아파트 전용면적 166㎡(55평)가 지난달 36억700...,중앙일보


In [20]:
data2.isnull().sum()

date        0
title       0
content     3
press      42
dtype: int64

In [21]:
data2.reset_index(inplace=True)
data2

,index,date,title,content,press
0,0,20230401,느긋한 강남 아파트마저 웃었다…서울은 재건축의 천국? [부동산360],"도곡한신재건축추진위, 안전진단 신청 접수서울 아파트, 7월부터 구청이 안전진단비 융...",헤럴드경제
1,1,20230401,"1·3 대책 후 서울 첫 분양 영등포자이, 예비당첨자 계약 ‘완판’","GS건설의 영등포자이 디그니티 모델하우스를 살펴보는 방문객들. 연합뉴스 ""입지 여건...",문화일보
2,2,20230401,"‘규제완화 후 서울 첫 분양’ 영등포자이, 예비당첨자 계약에서 ‘완판’",영등포자이 디그니티 견본주택. GS건설 제공헤럴드경제=안대용 기자 정부의 1·3 부...,헤럴드경제
3,3,20230401,서울 신축 아파트가 8억대?…아껴둔 통장 지를까요?[부동산360],‘휘경자이 디센시아’ 견본주택 가보니31일 강남구 대치동에 위치한 ‘휘경자이 디센시...,헤럴드경제
4,4,20230401,"규제완화 후 서울 첫 분양…영등포자이, 예비당첨자 계약서 '완판'",1·3 부동산 대책 발표 이후 서울에서 처음으로 입주자 모집공고를 내 눈길을 끌었던...,한국경제TV
...,...,...,...,...,...
26848,26859,20230630,"전국 아파트 가격, 14개월 만에 하락 멈춰","서울 아파트값은 6주 연속 상승세수도권 전셋값, 17개월 만에 반등지난해 5월부터 ...",동아일보
26849,26860,20230630,"압구정 현대는 10억까지? 고가 단지 재건축, 부담금 더 는다","정부, 野 반대에 재건축 초과이익환수법 수정안 국회 제출서울 강남구 압구정동 현대아...",조선일보
26850,26861,20230630,"[기업 브리핑] 태영건설, ‘대한민국 조경대상’ 국무총리상 외","태영건설, ‘대한민국 조경대상’ 국무총리상태영건설이 ‘대한민국 조경대상’에서 올해 ...",조선일보
26851,26862,20230630,"용산 집값, 송파 제치고 3위…흔들리는 ‘강남 3구 아성’",서울 용산구 서빙고동 신동아 아파트 전용면적 166㎡(55평)가 지난달 36억700...,중앙일보
